In [28]:
import numpy as np
from numpy import ndarray
import plotly.express as px

from optimizers import Adam
from losses import BinaryCrossentropy
from cifar_10_dataset_loading import load_cifar_10
from metrics import accuracy, nn_params_stats, activations_stats
from layers import Convolutional, Linear, Relu, BatchNorm, Flatten, Softmax, MaxPool, Layer

## Setup

### Data extraction

In [29]:
x_train, y_train, x_test, y_test = load_cifar_10()

In [30]:
classes = y_train.argmax(axis=1)

In [31]:
# Change range from [0, 255] to [-1, 1]
x_train = x_train / 255  
x_train.dtype

dtype('float64')

In [32]:
IMGS_IDX = [0, 1, 5673, 5494, 2, 55, 66, 776, 564]
x = x_train[IMGS_IDX]
y = y_train[IMGS_IDX]
px.imshow(x, facet_col=0, facet_col_wrap=4)

### Model declaration

In [33]:
def create_nn() -> list[Layer]:
    return [
        Convolutional((10, 7, 7, 3)),
        BatchNorm(),
        Relu(),
        MaxPool((2, 2)),
        Convolutional((20, 3, 3, 10)),
        BatchNorm(),
        Relu(),
        # MaxPool((2, 2)),
        Flatten(),
        Linear(2420, 128),
        Relu(),
        Linear(128, y.shape[1]),
        Softmax(),
    ]

## Training

In [34]:
training_stats = (
    Adam(
        create_nn(),
        x,
        y,
        BinaryCrossentropy(),
        starting_lr=0.015,
        lr_decay=0.0005,
        momentum_weight=0.9,
        ada_grad_weight=0.9,
    )
    .optimize_nn(
        epochs=10,
        batch_size=128,
        plt_x="epoch",
        plt_ys=["loss", "accuracy"]
    )
)

FigureWidget({
    'data': [{'hovertemplate': 'variable=loss<br>epoch=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'loss',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'loss',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'uid': '935c2c10-ed4b-4574-a918-47dc386dd6db',
              'x': array([0]),
              'xaxis': 'x2',
              'y': array([0.32508203]),
              'yaxis': 'y2'},
             {'hovertemplate': 'variable=accuracy<br>epoch=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'accuracy',
              'marker': {'color': '#EF553B', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'accuracy',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'uid': '49e053e8-1adc-4ef1-87be-3504b01c26a0',
     

In [35]:
N_SAMPLES_PER_CLASS = 500
NB_CLASSES = 10

class_masks = y_train.argmax(1, keepdims=True) == np.arange(NB_CLASSES)
take_first_n_of_class = lambda data, class_idx: data[class_masks[:, class_idx]][:N_SAMPLES_PER_CLASS]
take_first_n_of_each_class = lambda data: np.concatenate([take_first_n_of_class(data, class_idx) for class_idx in range(NB_CLASSES)], axis=0)
even_x_train = take_first_n_of_each_class(x_train)
even_y_train = take_first_n_of_each_class(y_train)
even_x_train.shape

(5000, 32, 32, 3)

In [36]:
bad_nn = create_nn()
bad_optimizer = Adam(
    bad_nn,
    even_x_train,
    even_y_train,
    BinaryCrossentropy(),
    starting_lr=0.025,
    lr_decay=0.0001,
    momentum_weight=0.85,
    ada_grad_weight=0.999,
)

In [37]:
bad_training_stats = (
    bad_optimizer
    .optimize_nn(
        epochs=100,
        batch_size=200,
        metrics=[accuracy, nn_params_stats, activations_stats],
        plt_x="epoch",
        plt_ys=[
            "loss",
            "accuracy",
            "learning_rate",
        ],
        height=850,
    )
)

Caught KeyboardInterrupt exception, returning training metrics.


In [ ]:
display(bad_training_stats.agg({"loss":"min", "accuracy":"max"}))

loss        0.096776
accuracy    0.790400
dtype: float64

In [ ]:
from copy import deepcopy

best_accuracy = 0
try:
    iteration = 0
    while True:
        starting_nn = create_nn()
        nn = deepcopy(starting_nn)
        optimizer = Adam(
            nn,
            even_x_train,   
            even_y_train,
            BinaryCrossentropy(),
            starting_lr=0.025,
            lr_decay=0.0005,
            momentum_weight=0.9,
            ada_grad_weight=0.999,
        )
        training_stats = (
            optimizer
            .optimize_nn(
                epochs=50,
                batch_size=100,
                metrics=[accuracy, nn_params_stats, activations_stats],
                catch_interrupt=False
            )
        )
        print(iteration, "accuracy:", training_stats["accuracy"].max())
        if training_stats["accuracy"].max() > best_accuracy:
            best_starting_nn, best_nn, best_optimizer, best_training_stats = starting_nn, nn, optimizer, training_stats
            best_accuracy = training_stats["accuracy"].max()
        if training_stats["accuracy"].max() > 50:
            break
        iteration += 1
except KeyboardInterrupt:
    print()
print("Best accuracy:", best_accuracy)

0 accuracy: 0.5978
Caught KeyboardInterrupt exception, returning training metrics.
1 accuracy: 0.2254


In [ ]:
nn = bad_nn

In [ ]:
px.imshow(x_train[5673])

In [38]:
img = x_train[[5673]]
convs = nn[1].forward(nn[0].forward(img))
convs.shape
px.imshow(convs[0], facet_col=2)

In [ ]:
reduce(lambda x, l: l.forward(x), nn, img) #.argmax(axis=1)

array([[0.34507341, 0.10575061, 0.0240158 , 0.03988019, 0.00964173,
        0.02572158, 0.00364022, 0.04975087, 0.24413487, 0.15239071]])

In [ ]:
y_train[5673]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
nn

In [44]:
from functools import reduce

img = x_train[[5673]]
convs = reduce(lambda x, l: l.forward(x), nn[:6], img) #nn[1].forward(nn[0].forward(img))
convs.shape
normed_convs = (convs - convs.min(axis=3, keepdims=True)) / (convs.max(axis=3, keepdims=True) - convs.min(axis=3, keepdims=True))
normed_convs.shape
px.imshow(normed_convs[0], facet_col=2)

In [62]:
def min_max_normalize(arr:np.ndarray, axis:tuple) -> np.ndarray:
    return (arr - arr.min(axis, keepdims=True)) / (arr.max(axis, keepdims=True) - arr.min(axis, keepdims=True))

normed_kernels = min_max_normalize(nn[0].kernels , (3))
normed_kernels.shape

px.imshow(normed_kernels, facet_col=0)

In [63]:
y_test_pred = reduce(lambda x, l: l.forward(x), nn, x_test)
accuracy({}, y_test_pred, y_test)

{'accuracy': np.float64(0.3358)}